In [1]:
import sqlite3

In [ ]:
# Set up database
conn = sqlite3.connect('sp500_stocks.db') # Create or connect to the db file
cursor = conn.cursor() # Create a cursor to excute command

In [ ]:
# Create neccessary tables
# Ticker info table
statement = """
    CREATE TABLE IF NOT EXISTS companies (
        ticker TEXT PRIMARY KEY,
        name TEXT NOT NULL,
        sector TEXT,
        industry TEXT,
        description TEXT,
        ipo_year INTEGER
    )
"""
cursor.execute(statement)


In [ ]:
# Price info table
statement = """
    CREATE TABLE IF NOT EXISTS price (
    ticker TEXT,
    date DATE,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    adj_close REAL,
    volume INTEGER,
    PRIMARY KEY (ticker, date),
    FOREIGN KEY (ticker) REFERENCES companies(ticker)
)
"""
cursor.execute(statement)

In [17]:
# Testing table insert and query works
insert_statment_1 = """
    INSERT INTO companies(ticker, name, sector, industry, description, ipo_year)
    VALUES ('AAPL', 'Apple Inc.', 'Techonology', 'Consumer Electronics', 'Technology powerhouse', 1980)"""
insert_statement_2 = """
    INSERT INTO price(ticker, date, open, high, low, close, adj_close, volume)
    VALUES ('AAPL', '2025-07-31', 208.49, 209.84, 207.16, 207.57, 207.57, 78744400)"""
cursor.execute(insert_statment_1)
cursor.execute(insert_statement_2)

In [18]:
cursor.execute("SELECT * FROM companies")
rows = cursor.fetchall()
print(rows)

[('AAPL', 'Apple Inc.', 'Techonology', 'Consumer Electronics', 'Technology powerhouse', 1980)]


In [19]:
cursor.execute("SELECT * FROM price")
rows = cursor.fetchall()
print(rows)

[('AAPL', '2025-07-31', 208.49, 209.84, 207.16, 207.57, 207.57, 78744400)]
